# Simulation data analysis

The purpose of this exercise is to carry out data analysis looping over the full range of VTK files in the simulation folder. We want to calculate some integrated quantities:

- Average density
- Standard deviation of density
- Speed
- RMS Mach number


## Loop over simulation files + Analysis

In [1]:
import pyvista as pv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

## Read  Units (units.out)

In [2]:
units_data = pd.read_csv("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/units.out")

rho_00   = np.array(units_data.loc[units_data["variable"] == "rho_0"]["normalisation"])
vel_00   = np.array(units_data.loc[units_data["variable"] == "v_0"]["normalisation"])
len_00   = np.array(units_data.loc[units_data["variable"] == "L_0"]["normalisation"])
c_iso    = np.array(units_data.loc[units_data["variable"] == "c_iso"]["normalisation"])

tim_00 = (len_00/vel_00)
bfi_00 = (np.sqrt(4*np.pi*rho_00*vel_00**2))

## Read  times (second column vtk.out)

In [3]:
time_data = pd.read_csv("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/vtk.out", sep='\s+', header=None)

time_code = time_data.iloc[:,1]

time_cgs = time_code*tim_00

print(len(time_cgs))

101


In [4]:
print(np.max(time_cgs))

6172000000000.0


In [5]:
if os.path.isdir("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/Figures1"):
    print("Directory already exists.")
else:
    print("Directory is being created.")
    os.mkdir("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/Figures1")

Directory already exists.


### I/O Function:

In [6]:
def io_simulation(filename):
    """
    I/O function:
    """
    
    # Opening the mesh
    mesh = pv.read(filename)
    
    # Getting the data arrays
    rho = pv.get_array(mesh, "rho", preference = 'cell') #density in code units
    vx1 = pv.get_array(mesh, "vx1", preference = 'cell') #vx in code units
    vx2 = pv.get_array(mesh, "vx2", preference = 'cell') #vy in code units    
    
    # Convert the arrays to 2D
    rho_2D = rho.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    vx1_2D = vx1.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    vx2_2D = vx2.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    
    # Convert arrays to CGS units
    rho_cgs2D = rho_2D*rho_00
    vx1_cgs2D = vx1_2D*vel_00
    vx2_cgs2D = vx2_2D*vel_00

    return rho_cgs2D, vx1_cgs2D, vx2_cgs2D

In [10]:
### Create empty lists

t_list = []

av_list = []

std_list = []

rms_mach_list = []


for i in range(0,len(time_cgs)):
    #print(i)
    
    filename = "/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/data.0{:03d}.vtk".format(i) 
    
    #print(filename)
    rho_cgs2D, vx1_cgs2D, vx2_cgs2D = io_simulation(filename)
    
    # Get i time
    time_i = time_cgs[i]
    print(time_i)
    
    
    
    # Diagnostics
    av_density  = np.mean(rho_cgs2D)
    std_density = np.std(rho_cgs2D)
    
    modulus_velocity = np.sqrt(vx1_cgs2D**2 + vx2_cgs2D**2)
    rms_mach = np.mean(modulus_velocity)/c_iso
    
    #print(av_density, std_density)
    
    # Fill up our empy lists
    
    t_list.append(time_i)
    av_list.append(av_density)
    std_list.append(std_density)
    rms_mach_list.append(rms_mach)
    
    # PLotting
    plt.figure(figsize = (16,10))
    
    plt.plot(t_list, std_list)
    
    plt.xlim(0, np.max(time_cgs))
    plt.ylim(1.e-26, 2.e-24)
    plt.savefig("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/std_dens{:03d}.png".format(i))
    plt.close()
    

    plt.figure(figsize = (16,10))
    
    plt.plot(t_list, rms_mach_list)
    
    plt.xlim(0, np.max(time_cgs))
    plt.ylim(0, 2)
    plt.savefig("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/Figures1/rms_mach{:03d}.png".format(i))
    plt.close()
    
# Outside the loop, we create a data frame:

df = pd.DataFrame({'time': t_list, 'Av. Dens.': av_list, 'Std. Dens.': std_list})

df.to_csv("/Users/wladimir/Dropbox/Sample_Data/TURB_DRIVE_SUP/datos1.csv", sep=',', float_format='{:.2e}'.format)
print(df)

0.0
56147733240.0
112343793240.0
182814856020.0
244348029580.0
296175794860.0
346957437000.0
415590694200.0
478889800400.0
553238021000.0
604333231600.0
666554398000.0
737688858200.0
791171398400.0
863854104799.9999
920062817400.0
983681324600.0
1047852842999.9999
1101169356400.0
1169717131400.0
1225532996200.0
1292360634800.0
1350463842800.0
1417110950400.0
1473747074000.0
1538223489200.0
1600975756200.0002
1659601424000.0002
1725210709800.0002
1783161469400.0
1847645291000.0002
1907673854400.0
1969693505000.0
2035666321600.0002
2096234009200.0
2159740494599.9998
2216392356800.0
2279749171200.0
2343710224400.0
2402558084200.0
2463908381400.0
2525413595800.0
2591891282000.0
2650990342200.0
2710559708800.0
2771972343200.0
2834253686600.0
2895335810400.0
2957889339000.0
3016529202400.0
3078682785400.0
3144948772000.0
3202860648000.0
3267459886000.0005
3329047188000.0005
3389702518000.0005
3454017844000.0
3515614404000.0
3576760408000.0
3638718030000.0
3696901474000.0
3763441806000.0
3825